# Estradiol Cypionate: Personal PK Fit & Schedule Simulator

This notebook helps you fit **your** estradiol kinetics from your injection history and lab levels, then visualize predictions and simulate schedules (e.g., 6‑day vs 7‑day).

> ⚠️ **Medical disclaimer:** This is an educational tool for personal data analysis and does **not** replace medical advice. Discuss results with your clinician.



## How to use

1. **Prepare your data** (CSV files):
   - `injections.csv` — columns: `date` (YYYY-MM-DD HH:MM), `dose_mg` (numeric), `route` (optional), `notes` (optional)
   - `labs.csv` — columns: `date` (YYYY-MM-DD HH:MM), `estradiol_pg_ml` (numeric), `assay_notes` (optional)

   You can start from the provided templates:
   - `/mnt/data/injections_template.csv`
   - `/mnt/data/labs_template.csv`

   Save your actual CSVs as `/mnt/data/injections.csv` and `/mnt/data/labs.csv`, or set custom paths below.

2. **Run the notebook top-to-bottom.**
   - The model supports **irregular injections** and estimates **your personal half‑life**.
   - You can choose a simple **instant-absorption model** or a more realistic **Bateman (absorption + elimination)** model for oil depot injections.

3. **Outputs you'll get:**
   - Fitted parameters (baseline, scale factor, absorption rate `ka`, elimination rate `ke`, and half-life `t½`).
   - Plots of predicted levels vs. labs.
   - Time-in-range stats and weekly/interval AUC.
   - Scenario simulator (try 6‑day, 7‑day, twice-weekly, micro-dosing, etc.).

---


In [ ]:

# If these imports fail locally, install with:
#   pip install numpy pandas matplotlib scipy
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Tuple, Dict, Optional, List

try:
    from scipy.optimize import least_squares
except Exception as e:
    raise RuntimeError("SciPy is required for fitting. Please install with: pip install scipy") from e

plt.rcParams.update({"figure.figsize": (10,6)})


In [ ]:

# === Configure file paths ===
INJ_CSV = "/mnt/data/estrogen_injections_08292025.csv"   # change if needed
LAB_CSV = "/mnt/data/labs.csv"         # change if needed

# === Load data ===
inj = pd.read_csv(INJ_CSV)
labs = pd.read_csv(LAB_CSV)

# Required columns
assert "date" in inj.columns and "dose_mg" in inj.columns, "injections.csv must have columns: date, dose_mg"
assert "date" in labs.columns and "estradiol_pg_ml" in labs.columns, "labs.csv must have columns: date, estradiol_pg_ml"

# Parse datetimes
inj["date"] = pd.to_datetime(inj["date"], errors="coerce")
labs["date"] = pd.to_datetime(labs["date"], errors="coerce")
if inj["date"].isna().any() or labs["date"].isna().any():
    bad_inj = inj[inj["date"].isna()]
    bad_labs = labs[labs["date"].isna()]
    raise ValueError(f"Found unparsable dates. Bad injections rows:\n{bad_inj}\n\nBad lab rows:\n{bad_labs}")

# Sort by time
inj = inj.sort_values("date").reset_index(drop=True)
labs = labs.sort_values("date").reset_index(drop=True)

inj.head(), labs.head()


In [ ]:

# Convert timestamps to a numeric time base (days)
t0 = min(inj["date"].min(), labs["date"].min()) - pd.Timedelta(days=7)  # pad one week before first event

def to_days(ts):
    return (ts - t0).dt.total_seconds() / (24*3600)

inj["t_days"] = to_days(inj["date"])
labs["t_days"] = to_days(labs["date"])

# Numpy-friendly version for scalars
def to_days_scalar(ts):
    return (pd.Timestamp(ts) - t0) / pd.Timedelta(days=1)

# Model components
def bateman_contrib(dt, ka, ke):
    """Bateman function contribution for a single dose over delta time dt (in days).

    Returns 0 for dt <= 0.
    Uses a numerically stable limit when ka ~ ke.
    """
    dt = np.asarray(dt)
    out = np.zeros_like(dt, dtype=float)
    mask = dt > 0
    dt_pos = dt[mask]

    # Avoid division by very small difference
    eps = 1e-8
    denom = (ka - ke)
    near = np.abs(denom) < eps

    if np.any(near):
        # limit as ka -> ke: ka * dt * exp(-ke*dt)
        out[mask][near] = ka * dt_pos[near] * np.exp(-ke * dt_pos[near])
        if np.any(~near):
            idx = np.where(near, False, True)
            dt_sel = dt_pos[idx]
            out[mask][idx] = (ka/ (ka - ke)) * (np.exp(-ke*dt_sel) - np.exp(-ka*dt_sel))
    else:
        out[mask] = (ka/ (ka - ke)) * (np.exp(-ke*dt_pos) - np.exp(-ka*dt_pos))
    return out

def instant_contrib(dt, ke):
    """Instantaneous absorption then mono-exponential elimination."""
    dt = np.asarray(dt)
    out = np.zeros_like(dt, dtype=float)
    mask = dt > 0
    dt_pos = dt[mask]
    out[mask] = np.exp(-ke * dt_pos)
    return out

def predict_timeseries(times, inj_t, doses, params, model="bateman"):
    """Compute predicted concentration at vector of times (days).

    params: dict with keys:
      - baseline >= 0
      - scale > 0 (converts mg * unit_response -> pg/mL)
      - ke > 0 (1/day)
      - ka > 0 (1/day) [bateman only]
      - tlag >= 0 (days) [optional; default 0]
    """
    baseline = params.get("baseline", 0.0)
    scale = params.get("scale", 1.0)
    ke = max(params.get("ke", 0.1), 1e-6)
    tlag = max(params.get("tlag", 0.0), 0.0)
    if model == "bateman":
        ka = max(params.get("ka", 0.5), 1e-6)

    times = np.asarray(times)
    pred = np.full_like(times, fill_value=baseline, dtype=float)

    for t_i, d in zip(inj_t, doses):
        dt = times - (t_i + tlag)
        if model == "bateman":
            pred += scale * d * bateman_contrib(dt, ka=ka, ke=ke)
        else:
            pred += scale * d * instant_contrib(dt, ke=ke)
    return pred

def half_life(ke):
    return math.log(2)/ke


In [ ]:

# === Model selection ===
# Choose "bateman" for depot absorption (recommended for estradiol cypionate) or "instant" for simple decay
MODEL = "bateman"  # "bateman" or "instant"

# === Initial guesses & bounds ===
# Reasonable starting points (you can adjust these if fit struggles)
init = {
    "baseline": 30.0,  # pg/mL
    "scale": 15.0,     # pg/mL per mg * unit_response
    "ke": math.log(2)/5.0,   # elimination half-life ~5 days -> 0.1386 /day
    "ka": math.log(2)/1.5,   # absorption half-life ~1.5 days -> ~0.462 /day (bateman only)
    "tlag": 0.0,             # optional lag (days)
}

# Parameter bounds to keep fit sensible
bounds = {
    "baseline": (0.0, 500.0),
    "scale": (1e-6, 1e6),
    "ke": (math.log(2)/60.0, math.log(2)/0.5),  # half-life between ~0.5 d and 60 d
    "ka": (math.log(2)/60.0, math.log(2)/0.2),  # half-life between ~0.2 d and 60 d
    "tlag": (0.0, 2.0),
}

# Prepare data arrays
t_lab = labs["t_days"].to_numpy()
y_lab = labs["estradiol_pg_ml"].to_numpy()
t_inj = inj["t_days"].to_numpy()
doses = inj["dose_mg"].to_numpy()

# Assemble x0 and bounds vectors
key_order = ["baseline", "scale", "ke"] + (["ka"] if MODEL == "bateman" else []) + ["tlag"]
x0 = np.array([init[k] for k in key_order], dtype=float)
lb = np.array([bounds[k][0] for k in key_order], dtype=float)
ub = np.array([bounds[k][1] for k in key_order], dtype=float)

def vec_to_params(x):
    d = dict(zip(key_order, x))
    return d

def residuals(x):
    p = vec_to_params(x)
    pred = predict_timeseries(t_lab, t_inj, doses, p, model=MODEL)
    # Use a robust residual (Huber via least_squares 'soft_l1')
    return pred - y_lab

res = least_squares(residuals, x0, bounds=(lb, ub), loss="soft_l1", f_scale=25.0, max_nfev=10000)

params_fit = vec_to_params(res.x)
pred_lab = predict_timeseries(t_lab, t_inj, doses, params_fit, model=MODEL)

print("=== Fit Results ===")
for k in key_order:
    print(f"{k:9s}: {params_fit[k]:.6g}")
print(f"half_life_days (ke): {half_life(params_fit['ke']):.3f}")

# Approximate parameter uncertainty using Jacobian
if res.jac is not None and res.jac.size > 0 and len(y_lab) > len(res.x):
    J = res.jac
    dof = max(len(y_lab) - len(res.x), 1)
    sigma2 = (res.fun**2).sum() / dof
    try:
        cov = np.linalg.inv(J.T @ J) * sigma2
        se = np.sqrt(np.diag(cov))
    except np.linalg.LinAlgError:
        cov = None
        se = np.full_like(res.x, np.nan, dtype=float)
else:
    cov = None
    se = np.full_like(res.x, np.nan, dtype=float)

print("\n=== Std. Errors (approximate) ===")
for k, s in zip(key_order, se):
    print(f"{k:9s}: {s:.3g}")

# Propagate to half-life (delta method on ke)
ke_idx = key_order.index("ke")
if not math.isnan(se[ke_idx]):
    ke = params_fit["ke"]
    dtdke = -math.log(2)/(ke**2)
    var_t12 = (dtdke**2) * (se[ke_idx]**2)
    se_t12 = math.sqrt(max(var_t12, 0.0))
    print(f"half_life_days SE (approx): {se_t12:.3f}")
else:
    print("half_life SE not available.")


In [ ]:

# === Plot: Predicted curve vs Labs and Injections ===
t_min = min(inj["t_days"].min(), labs["t_days"].min()) - 3
t_max = max(inj["t_days"].max(), labs["t_days"].max()) + 7
times_dense = np.linspace(t_min, t_max, 800)
pred_dense = predict_timeseries(times_dense, t_inj, doses, params_fit, model=MODEL)

plt.figure()
plt.plot(times_dense, pred_dense, label="Predicted")
plt.scatter(t_lab, y_lab, marker="o", label="Labs")
# Draw injection stems
for ti, d in zip(t_inj, doses):
    plt.vlines(ti, ymin=0, ymax=max(pred_dense.max(), y_lab.max())*1.05, linestyles="dotted")
    plt.text(ti, 0, f"{d:.2f} mg", rotation=90, va="bottom", ha="center", fontsize=8)

plt.title("Estradiol (pg/mL) — Model vs Labs")
plt.xlabel(f"Days since {pd.to_datetime(t0).date()}")
plt.ylabel("Estradiol (pg/mL)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

# === Metrics: Time-in-Range and AUC over windows ===
LOW, HIGH = 100, 200  # adjust to your target range (pg/mL)

def time_in_range(times, conc, low=LOW, high=HIGH):
    within = (conc >= low) & (conc <= high)
    # Approximate proportion of time using trapezoidal integral of indicator
    return np.trapz(within.astype(float), times) / (times[-1] - times[0])

def auc(times, conc, start=None, end=None):
    if start is not None or end is not None:
        mask = np.ones_like(times, dtype=bool)
        if start is not None:
            mask &= times >= start
        if end is not None:
            mask &= times <= end
        return np.trapz(conc[mask], times[mask])
    return np.trapz(conc, times)

tir = time_in_range(times_dense, pred_dense, LOW, HIGH)
weekly_auc = auc(times_dense, pred_dense, start=times_dense[-1]-7, end=times_dense[-1])

print(f"Time in {LOW}-{HIGH} pg/mL range: {100*tir:.1f}% of time over plotted interval")
print(f"AUC over last 7 days (pg·day/mL): {weekly_auc:.1f}")


In [ ]:

# === Scenario Simulator ===
# Try alternative schedules and overlay predictions.
# Define a helper to build a hypothetical schedule starting from last injection date.

def simulate_schedule(start_ts, dose_mg, every_days, n_doses=10):
    times = [start_ts + i*every_days for i in range(n_doses)]
    return np.array(times), np.full(n_doses, dose_mg, dtype=float)

last_inj_t = float(inj["t_days"].max())
sim_starts_at = last_inj_t  # start right after your last recorded injection in the dataset

scenarios = {
    "Every 7 days": simulate_schedule(sim_starts_at, dose_mg=float(doses[-1]), every_days=7, n_doses=12),
    "Every 6 days": simulate_schedule(sim_starts_at, dose_mg=float(doses[-1]), every_days=6, n_doses=12),
    "Twice weekly (3.5d)": simulate_schedule(sim_starts_at, dose_mg=float(doses[-1]/2), every_days=3.5, n_doses=24),
}

t_future = np.linspace(sim_starts_at, sim_starts_at + 56, 1200)  # simulate 8 weeks ahead

plt.figure()
# Plot current model continuation (no new doses) as baseline
base_pred = predict_timeseries(t_future, t_inj, doses, params_fit, model=MODEL)
plt.plot(t_future, base_pred, label="No new doses (baseline)")
for name, (t_sched, d_sched) in scenarios.items():
    pred_scn = predict_timeseries(t_future, np.concatenate([t_inj, t_sched]), np.concatenate([doses, d_sched]), params_fit, model=MODEL)
    plt.plot(t_future, pred_scn, label=name)

plt.title("Schedule Scenarios — Predicted Estradiol")
plt.xlabel(f"Days since {pd.to_datetime(t0).date()}")
plt.ylabel("Estradiol (pg/mL)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

# === Data Quality Checks ===
# Useful quick checks for irregularities or outliers
print("Dose summary (mg):")
print(inj["dose_mg"].describe())

print("\nIntervals between injections (days):")
if len(inj) > 1:
    intervals = np.diff(inj["t_days"])
    print(pd.Series(intervals).describe())
else:
    print("Only one injection in data.")

print("\nLabs summary (pg/mL):")
print(labs["estradiol_pg_ml"].describe())
